<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_EQDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Setup and Initial Data
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
data = [
    {'Position': 1, 'Bucket': 6, 'Market Cap': 'Large', 'Economy': 'Advanced', 'Sector': 'Industrials', 'Underlying': 'Equity A', 'Sensitivity': -5829},
    {'Position': 2, 'Bucket': 6, 'Market Cap': 'Large', 'Economy': 'Advanced', 'Sector': 'Industrials', 'Underlying': 'Equity B', 'Sensitivity': -5852},
    {'Position': 3, 'Bucket': 2, 'Market Cap': 'Large', 'Economy': 'Emerging', 'Sector': 'Industrials', 'Underlying': 'Equity C', 'Sensitivity': -2091}
]

# --- Regulatory Parameters ---
# Risk Weights for Equity Buckets as per Article 325ap
risk_weights = {
    2: 0.60,  # Large Cap, Emerging, Industrials
    6: 0.35   # Large Cap, Advanced, Industrials
}

# Intra-Bucket Correlation for Equity as per Article 325aq
# For Bucket 6 (Large Cap, Advanced Economy)
rho_kl_medium = 0.25

# Cross-Bucket Correlation for Equity as per Article 325ar
# For Bucket 2 vs Bucket 6 (both fall between buckets 1-10)
gamma_bc_medium = 0.15

# Create an initial DataFrame
portfolio_df = pd.DataFrame(data)

print("--- Initial Setup Complete ---")
print(f"Loaded {len(portfolio_df)} positions.")
# ---
# Expected output for Cell 1:
# --- Initial Setup Complete ---
# Loaded 3 positions.
# ---

--- Initial Setup Complete ---
Loaded 3 positions.


In [ ]:
# Cell 2: Step 1 & 2 - Identify Risk Factors and Net Sensitivities
# In this portfolio, each position has a unique underlying equity.
# Therefore, no netting occurs, and the net sensitivity for each risk factor
# is the initial sensitivity of the position.

print("### Step 1 & 2: Net Sensitivities (sₖ) ###")
print("Since each position has a unique underlying, the net sensitivities are the same as the initial sensitivities.")

# For clarity, we rename the 'Sensitivity' column to 'Net_Sensitivity_sk'
net_sensitivities_df = portfolio_df.rename(columns={'Sensitivity': 'Net_Sensitivity_sk'})

# Display the DataFrame, sorted by Bucket
display(net_sensitivities_df.sort_values('Bucket').set_index('Position'))
# ---
# Expected output for Cell 2:
# ### Step 1 & 2: Net Sensitivities (sₖ) ###
# Since each position has a unique underlying, the net sensitivities are the same as the initial sensitivities.
# (DataFrame showing the initial positions as net sensitivities)
# ---

### Step 1 & 2: Net Sensitivities (sₖ) ###
Since each position has a unique underlying, the net sensitivities are the same as the initial sensitivities.


,Bucket,Market Cap,Economy,Sector,Underlying,Net_Sensitivity_sk
Position,,,,,,
3,2,Large,Emerging,Industrials,Equity C,-2091
1,6,Large,Advanced,Industrials,Equity A,-5829
2,6,Large,Advanced,Industrials,Equity B,-5852


In [ ]:
# Cell 3: Step 3 - Calculate Weighted Sensitivities (WSₖ)
# Each net sensitivity is multiplied by its regulatory risk weight (RWₖ) from Article 325ap.

print("\n### Step 3: Calculate Weighted Sensitivities (WSₖ) ###")
print("Applying regulatory risk weights to the net sensitivities.")

# Map the risk weights to the DataFrame
net_sensitivities_df['Risk_Weight_RWk'] = net_sensitivities_df['Bucket'].map(risk_weights)

# Calculate the weighted sensitivity
net_sensitivities_df['Weighted_Sensitivity_WSk'] = net_sensitivities_df['Net_Sensitivity_sk'] * net_sensitivities_df['Risk_Weight_RWk']

# Display the results
display(net_sensitivities_df[['Underlying', 'Bucket', 'Net_Sensitivity_sk', 'Risk_Weight_RWk', 'Weighted_Sensitivity_WSk']].sort_values('Bucket'))
# ---
# Expected output for Cell 3:
# ### Step 3: Calculate Weighted Sensitivities (WSₖ) ###
# Applying regulatory risk weights to the net sensitivities.
# (DataFrame showing sensitivities, risk weights, and weighted sensitivities)
# ---


### Step 3: Calculate Weighted Sensitivities (WSₖ) ###
Applying regulatory risk weights to the net sensitivities.


,Underlying,Bucket,Net_Sensitivity_sk,Risk_Weight_RWk,Weighted_Sensitivity_WSk
2,Equity C,2,-2091,0.60,-1254.60
0,Equity A,6,-5829,0.35,-2040.15
1,Equity B,6,-5852,0.35,-2048.20


In [ ]:
# Cell 4: Step 6 - Intra-Bucket Aggregation (Kₑ)
# This function calculates the bucket-specific capital (Kₑ) for a given scenario.

def calculate_kb(bucket_df, scenario='medium'):
    """Calculates the bucket-specific capital (Kₑ) based on Article 325f(7)."""

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        rho_kl = min(rho_kl_medium * 1.25, 1.0)
    elif scenario == 'low':
        rho_kl = max(2 * rho_kl_medium - 1, 0.75 * rho_kl_medium)
    else: # medium
        rho_kl = rho_kl_medium

    ws = bucket_df['Weighted_Sensitivity_WSk'].values

    # If only one sensitivity in the bucket, Kb is its absolute value
    if len(ws) == 1:
        return np.abs(ws[0])

    # Sum of squares of weighted sensitivities
    sum_ws_sq = np.sum(ws**2)

    # Sum of cross-products
    # WSk * WSl for all pairs (k, l) where k != l
    cross_product_sum = rho_kl * (np.sum(ws)**2 - sum_ws_sq)

    k_squared = max(sum_ws_sq + cross_product_sum, 0)
    return np.sqrt(k_squared)

print("\n### Step 6: Intra-Bucket Aggregation (Medium Scenario) ###")
print("Calculating bucket-specific capital (Kₑ) for the Medium Correlation Scenario.")

# Calculate Kₑ for each bucket for all scenarios to use later
bucket_capital = {}
for scenario in ['medium', 'high', 'low']:
    scenario_kbs = {}
    for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
        scenario_kbs[bucket_id] = calculate_kb(group, scenario=scenario)
    bucket_capital[scenario] = scenario_kbs

# Calculate Sₑ for each bucket (sum of weighted sensitivities)
s_b_values = net_sensitivities_df.groupby('Bucket')['Weighted_Sensitivity_WSk'].sum().to_dict()

# Display results for the medium scenario
kb_results_df = pd.DataFrame({
    'Bucket': list(bucket_capital['medium'].keys()),
    'Kb_Medium': list(bucket_capital['medium'].values()),
    'Sb': list(s_b_values.values())
})
display(kb_results_df)
# ---
# Expected output for Cell 4:
# ### Step 6: Intra-Bucket Aggregation (Medium Scenario) ###
# Calculating bucket-specific capital (Kₑ) for the Medium Correlation Scenario.
# (DataFrame showing Kb and Sb for each bucket)
# ---


### Step 6: Intra-Bucket Aggregation (Medium Scenario) ###
Calculating bucket-specific capital (Kₑ) for the Medium Correlation Scenario.


,Bucket,Kb_Medium,Sb
0,2,1254.600000,-1254.60
1,6,3232.128227,-4088.35


In [ ]:
# Cell 5: Step 7, 8 & 9 - Across-Bucket Aggregation and Final Charge
# This function aggregates the bucket-level capital (Kₑ) to get the final
# risk class capital for a given scenario.

def calculate_final_capital(kb_dict, sb_dict, scenario='medium'):
    """Calculates the final risk class capital based on Article 325f(8)."""

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        gamma = min(gamma_bc_medium * 1.25, 1.0)
    elif scenario == 'low':
        gamma = max(2 * gamma_bc_medium - 1, 0.75 * gamma_bc_medium)
    else: # medium
        gamma = gamma_bc_medium

    k_values = list(kb_dict.values())
    s_values = list(sb_dict.values())

    sum_k_sq = np.sum(np.array(k_values)**2)

    # Sum of cross-bucket terms: gamma * Sb * Sc
    cross_bucket_sum = gamma * (np.sum(s_values)**2 - np.sum(np.array(s_values)**2))

    final_capital_sq = max(sum_k_sq + cross_bucket_sum, 0)
    return np.sqrt(final_capital_sq)

print("\n### Step 7, 8 & 9: Final Capital Charge Across Scenarios ###")

# Calculate final capital for all scenarios
scenario_results = {}
for scenario in ['medium', 'high', 'low']:
    scenario_results[scenario.capitalize()] = calculate_final_capital(
        bucket_capital[scenario],
        s_b_values,
        scenario=scenario
    )

final_charge = max(scenario_results.values())
final_scenario = max(scenario_results, key=scenario_results.get)

# Display Summary
summary_df = pd.DataFrame(list(scenario_results.items()), columns=['Scenario', 'Capital Charge'])
print("Final capital charge for each scenario:")
display(summary_df.set_index('Scenario'))

print(f"\nThe highest requirement is from the {final_scenario} Correlation Scenario.")
print(f"\nFinal Equity Delta Capital Requirement: {final_charge:,.2f}")
# ---
# Expected output for Cell 5:
# ### Step 7, 8 & 9: Final Capital Charge Across Scenarios ###
# Final capital charge for each scenario:
# (DataFrame showing capital for Low, Medium, High scenarios)
#
# The highest requirement is from the High Correlation Scenario.
#
# Final Equity Delta Capital Requirement: 3,803.56
# ---


### Step 7, 8 & 9: Final Capital Charge Across Scenarios ###
Final capital charge for each scenario:


,Capital Charge
Scenario,
Medium,3682.315469
High,3803.481288
Low,3557.024671



The highest requirement is from the High Correlation Scenario.

Final Equity Delta Capital Requirement: 3,803.48
